In [1]:
import pandas as pd
import numpy as np
import ast
import json
import os
from essentia.standard import MonoLoader

2023-03-28 17:34:40.342807: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
[   INFO   ] MusicExtractorSVM: no classifier models were configured by default
2023-03-28 17:34:40.763114: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-28 17:34:40.764662: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2023-03-28 17:34:40.768731: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:65:00.0 name: NVIDIA GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.65GHz coreCount: 68 deviceMemorySize: 10.76GiB dev

In [2]:
from tqdm.notebook import tqdm

In [3]:
tqdm.pandas()

In [4]:

args = pd.Series({
    "root_dir":"/mnt/disks/data/",
    "dataset_path":"/mnt/disks/data/fma/fma_large"
})


In [6]:


base_path = os.path.join(args.root_dir,"fma")


# In[17]:

metadata_path_fma = os.path.join(base_path,"fma_metadata")



In [7]:
# Cria um dicionário que associa o ID de cada música aos IDs de seus gêneros musicais
tracks_df = pd.read_csv(os.path.join(metadata_path_fma,'tracks.csv'), header=[0,1], index_col=0)

In [8]:
tracks_df.columns = ['_'.join(col) for col in tracks_df.columns.values]

In [9]:
tracks_df.track_genres

track_id
2                  [21]
3                  [21]
5                  [21]
10                 [10]
20            [76, 103]
              ...      
155316             [25]
155317             [25]
155318             [25]
155319             [25]
155320    [10, 12, 169]
Name: track_genres, Length: 106574, dtype: object

In [10]:
tracks_df.track_genres.value_counts()

[21]                    2735
[15]                    2689
[]                      2231
[12]                    1896
[1, 30, 38, 41, 247]    1633
                        ... 
[12, 542]                  1
[92, 602]                  1
[102, 189]                 1
[14, 30, 92]               1
[10, 12, 169]              1
Name: track_genres, Length: 4768, dtype: int64

In [11]:
tracks_df.reset_index(inplace=True)

In [13]:
def find_path(track_id,dataset_path):
    track_id = track_id.zfill(6)
    folder_id = track_id[0:3]
    file_path = os.path.join(dataset_path,folder_id,track_id+'.mp3')
    return file_path
    


# In[36]:


tracks_df['file_path'] = tracks_df.track_id.apply(lambda x: find_path(str(x),args.dataset_path))



In [14]:
## Fazer o valid genre aqui

In [15]:
def valid_music(file_path):
    try:
        # we start by instantiating the audio loader:
        loader = MonoLoader(filename=file_path)

        return True
    except:
        return False

In [19]:
valid_music('/mnt/disks/data/fma/fma_large/000/000003.mp3')

True

In [ ]:
tracks_df.loc[:,'valid'] = tracks_df.file_path.progress_apply(lambda x: valid_music(x))

  0%|          | 0/106574 [00:00<?, ?it/s]

In [ ]:
tracks_df_filter = tracks_df[[]]

In [138]:
valid_df = pd.read_csv(os.path.join(train_path,'tracks_valid.csv'))

In [141]:
valid_df

,track_id,valid_genre
0,2.0,['21']
1,3.0,['21']
2,5.0,['21']
3,10.0,['10']
4,20.0,"['76', '103']"
...,...,...
104181,155316.0,['25']
104182,155317.0,['25']
104183,155318.0,['25']
104184,155319.0,['25']


In [140]:
valid_df = valid_df[['track_id','valid_genre']]

In [142]:
df_merge = pd.merge(tracks_df,valid_df,on='track_id')

In [143]:
df_merge.valid_genre.values

array(["['21']", "['21']", "['21']", ..., "['25']", "['25']",
       "['10', '12', '169']"], dtype=object)

In [144]:
df_merge.to_csv(os.path.join(metadata_path_fma,'tracks_valid.csv'),index=False)